In [44]:
import pandas as pd
from os import path
import os

#DATA_IN = "../../data/raw/semi-raw_data.csv"
DATA_IN = "../../data/raw/Jefferson_County_KY_Signalized_Intersections.csv"
CSV_OUT = "../../data/clean/Jefferson_County_KY_Signalized_Intersections.csv"
assert path.exists(DATA_IN)

df=pd.read_csv(DATA_IN)

In [45]:
df.columns



Index(['X', 'Y', 'OBJECTID', 'UNITID', 'SIGID', 'MAINSTREET', 'CROSSSTREET',
       'OWNER2', 'ROUTE', 'MILEPOINT', 'DESCRIPTION', 'TIMES', 'OWNER', 'TYPE',
       'INTID', 'ATMSID'],
      dtype='object')

In [62]:
keepers = ['X', 'Y', 'MAINSTREET', 'CROSSSTREET', 'DESCRIPTION', 'INTID', 'OBJECTID']
dropping = set(df.columns).difference(keepers)

renames = {'X':"longitude",
        'Y':'latitude',
        'MAINSTREET':'main_street', 
        'CROSSSTREET':'cross_street', 
        'DESCRIPTION':'description', 
        'INTID':'intid', 
        'OBJECTID':'objectid'}

dropped = df.drop(dropping, axis=1)
renamed = dropped.rename(renames, axis=1)

renamed.intid = renamed.intid.str.strip()


In [75]:

int_id_counts = renamed.intid.value_counts()
repeat_ids = int_id_counts[int_id_counts > 1]
id_2 = '2'
id_not2 = repeat_ids.index[1:]

discard = list()
for id in id_not2:
    D = renamed[renamed.intid == id]
    discard.extend(D.index[1:])
discard

filtered = renamed.drop(discard)
filtered.intid.value_counts()

intid
2                23
1604203842663     1
244589126D886     1
2785299966026     1
2785199966026     1
                 ..
375906564154      1
418544927132      1
436629017132      1
508029305629      1
1217075539999     1
Name: count, Length: 1048, dtype: int64

In [91]:
assert not bool(set(filtered.objectid) & set(filtered.intid))
id2 = filtered[filtered.intid == '2']
filtered.loc[id2.index, 'intid'] = filtered.objectid

AssertionError: 

In [93]:
filtered.intid.value_counts()

intid
76747569996      1
1604604172663    1
2785199966026    1
2775445534815    1
379827122930     1
                ..
418544927132     1
436629017132     1
508029305629     1
529007234084     1
1217075539999    1
Name: count, Length: 1070, dtype: int64

In [48]:

# Deal with duplicated intids != '2'
duplicate_intids = renamed[(renamed['intid ct'] > 1) & (renamed['intid'] != '2')].intid.unique()

duplicates = list()
for id in duplicate_intids:
    u = renamed[renamed.intid==id]
    duplicates.append(u.iloc[1].name)

U = renamed.loc[duplicates].intid.apply(lambda x:f"{x}_2")
#renamed.intid.update(U)

In [49]:
renamed.intid.value_counts()

intid
2                23
2183216123540     3
1606703843932     3
1674018857012     2
1379703840617     2
                 ..
418544927132      1
436629017132      1
508029305629      1
529007234084      1
1217075539999     1
Name: count, Length: 1048, dtype: int64